# Análisis de Calidad del Aire en Colombia
Este cuaderno detalla el proceso de carga, limpieza y análisis de datos relacionados con la calidad del aire en diferentes regiones de Colombia.

## 1. Carga de Datos
Se comienza cargando los datos de sensores de calidad del aire proporcionados en un archivo CSV.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pancachocaliente/calidad-del-aire-en-colombia")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Cargar los datos
file_path = './data/DATOS_DE_CALIDAD_DEL_AIRE_EN_COLOMBIA_2011-2018_20240311.csv'
data = pd.read_csv(file_path)

# Mostrar las primeras filas
data.head()

## 2. Exploración Inicial de los Datos
Se examinan las dimensiones del conjunto de datos y se identifican valores nulos, duplicados y tipos de datos.

In [ ]:
# Dimensiones y tipos de datos
print(data.info())

# Valores nulos
print(data.isnull().sum())

# Duplicados
print(f"Duplicados: {data.duplicated().sum()}")

## 3. Limpieza de Datos
En esta sección se manejan valores nulos, se corrigen inconsistencias en los datos y se transforman columnas clave para el análisis.

In [ ]:
# Eliminar duplicados
data = data.drop_duplicates()

# Manejo de valores nulos
data = data.dropna()

# Corrección de formatos de columnas
data['Fecha'] = pd.to_datetime(data['Fecha'], errors='coerce')

# Filtrar filas con fechas válidas
data = data[data['Fecha'].notnull()]

# Inspeccionar nuevamente
print(data.info())

## 4. Transformación de Datos
Se generan columnas adicionales para facilitar el análisis, como mes, año y región.

In [ ]:
# Agregar columnas adicionales
data['Año'] = data['Fecha'].dt.year
data['Mes'] = data['Fecha'].dt.month
data['Región'] = data['Departamento'] + ' - ' + data['Nombre del municipio']

# Verificar las nuevas columnas
data.head()

## 5. Análisis de Datos
Se realiza un análisis exploratorio para identificar patrones, tendencias y posibles problemas de calidad del aire por región y año.

In [ ]:
# Análisis por regiones
plt.figure(figsize=(12, 6))
sns.countplot(data=data, y='Región', order=data['Región'].value_counts().index)
plt.title('Número de registros por Región')
plt.show()

# Análisis por años
plt.figure(figsize=(12, 6))
sns.countplot(data=data, x='Año', palette='viridis')
plt.title('Número de registros por Año')
plt.xticks(rotation=45)
plt.show()

## 6. Generación de Informe con Sweetviz
Se utiliza la biblioteca Sweetviz para generar un informe detallado de los datos en formato HTML.

In [ ]:
import sweetviz as sv

# Crear el informe
reporte = sv.analyze(data)

# Guardar el informe como HTML
reporte.show_html('./report/informe_calidad_aire.html')

## 7. Conclusiones
Se detallan los principales hallazgos del análisis de calidad del aire en las diferentes regiones de Colombia.